In [26]:
library(readstata13)
library(dplyr)
library(lme4)
library(nlme)
library(car)
library(psych)

In [27]:
df = read.dta13("/Users/macintoshhd/Desktop/dataset/Western States Survey/data.dta",  
           missing.type=TRUE, generate.factors=TRUE)

In [28]:
zero.one<-function(x){
  min.x<-min(x, na.rm=T)
  max.x<-max(x-min.x, na.rm=T)
  return((x-min.x)/max.x)
}

state_list = c("Washington", "Oregon", "California", 
         "Idaho", "Utah", "Nevada", "Arizona",
         "New Mexico", "Colorado", "Wyoming", 
         "Montana", "Texas", "Oklahoma", "Kansas",
         "Sonora, Mexico", "Chihuaha, Mexico", 
         "Coahila, Mexico", "British Columbia, Canada",
         "Alberta, Canada")

region_list = c("West", "Southwest", "Northwest", 
           "West Coast", "Intermountain West",
          "Rocky Mountains", "Other")

state = ifelse(df$WSS01_1 == "selected", "Washington", NA) 
region = ifelse(df$WSS01_b_1 == "selected", "West", NA) 

opt = paste0("WSS01_", c(2:19))
for (i in 1:length(opt)){
    state = ifelse(df[, opt[i]] == "selected", state_list[i+1] , state) 
}

opt = paste0("WSS01_b_", c(2:7))
for (i in 1:length(opt)){
    region = ifelse(df[, opt[i]] == "selected", region_list[i+1] , region) 
}

df$vote1 <- car::recode(as.numeric(df$WSS36_b), "1=1 ; 2=0; else=NA")
df$vote2 <- car::recode(as.numeric(df$WSS36_c), "1=1 ; 2=0; else=NA")
df$voted <- ifelse(is.na(df$WSS36_b), df$vote2, df$vote1)

                  
df$state = state
df$region = region


In [41]:
agree_key <- list(`1`= 5, `2`= 4,`3`= 3, `4`= 2, `5`= 1)
disagree_key <- list(`1`= 5, `2`= 4,`3`= 3, `4`= 2, `5`= 1)

df <-  df %>%
       mutate(state  =  state) %>%
       mutate(region =  region) %>%
       mutate(state_residence =  inputstate) %>%
       mutate(state_identity_identity_strength = state) %>%
    
       mutate(age             = 2020 - birthyr) %>%
       mutate(gen             = car::recode(age, "18:39 = 'Millenial'; 40:59 = 'Gen X'; 60:80 = 'Boomer'; 80:92 = 'Greatest'" )) %>%
       mutate(female          = car::recode(as.numeric(gender), "1=0; 2=1")) %>%
       mutate(white           = car::recode(as.numeric(race),  "1=1 ;  2:8=0"))  %>%
       mutate(latino          = car::recode(as.numeric(race), "3=1;  1:2=0; 4:8=0"))  %>%
       mutate(black           = car::recode(as.numeric(race), " 2=1;  1=0; 3:8=0"))  %>%
       mutate(other           = car::recode(as.numeric(race), " 1:3=0; 4:8=1"))   %>%
      # Always coded in conservative, republican direction
       mutate(pid3            = car::recode(as.numeric(pid3), "1=1; 2=3; 3=2")) %>%
       mutate(pid7            = car::recode(as.numeric(pid7), "8:9 = NA"))  %>%
       mutate(vote2016        = car::recode(as.numeric(presvote16post), "1=0; 2=1; else=NA")) %>%
       mutate(married         = car::recode(as.numeric(marstat), "1=1; 2:6=0")) %>%
       mutate(college         = car::recode(as.numeric(educ), "1:3=0; 4:8=1"))   %>%
       mutate(income          = car::recode(as.numeric(faminc_new), "1:7=0; 8:17=1"))  %>% 
       mutate(ideology        = dplyr::recode(as.numeric(ideo5), `1`=1, `2`=2, `3`=3, `4`=4, `5`=5)) %>%
       mutate(christian       = dplyr::recode(as.numeric(religpew), `1`=1, `2`=1, `3`=1, `4`=1, `5`=0, `6`=0, `7`=0, `8`=0, `9`=0, `10`=0, `11`=0, `12`=0)) %>%
       mutate(interest        = dplyr::recode(as.numeric(newsint), `1`=4, `2`=3, `3`=2, `4`=1)) %>%
       mutate(auth1           = dplyr::recode(as.numeric(WSS07_1a), `1`= 0, `2`=1)) %>% 
       mutate(auth2           = dplyr::recode(as.numeric(WSS07_1b), `1`= 1, `2`=0)) %>% 
       mutate(auth3           = dplyr::recode(as.numeric(WSS07_1c), `1`= 0, `2`=1)) %>% 
       mutate(auth4           = dplyr::recode(as.numeric(WSS07_1d), `1`= 0, `2`=1)) %>%
       mutate(az_t1           = dplyr::recode(as.numeric(WSS28_split), `1`= 0, `2`=1)) %>%
       mutate(az_rep_state    = as.numeric(WSS28_1)) %>%
       mutate(az_rep_nat      = as.numeric(WSS28_2)) %>%
       mutate(az_dem_state    = as.numeric(WSS28_3)) %>%
       mutate(az_dem_nat      = as.numeric(WSS28_4)) %>% 
       mutate(race            = car::recode(as.numeric(race), "1=1; 2=2; 3=3; 4:8=NA")) 

              


In [42]:

df$authoritarianism = rowMeans(cbind(df$auth1, df$auth2, 
                                df$auth3, df$auth4), na.rm=T)

### m0 is a intercept-only model

In [43]:
df1 <- df %>% 
  select(pid7, age, female, race, college, authoritarianism, state) %>% na.omit()
# race: 1-white, 2-black, 3-latino
m0 <- lme(pid7 ~ 1,
          data = df1,  
          method = "ML", 
          na.action = "na.omit",
          random = ~ 1|state)
summary(m0)



Linear mixed-effects model fit by maximum likelihood
  Data: df1 
      AIC      BIC    logLik
  13942.8 13960.99 -6968.402

Random effects:
 Formula: ~1 | state
        (Intercept) Residual
StdDev:    0.181639 2.179688

Fixed effects:  pid7 ~ 1 
               Value  Std.Error   DF  t-value p-value
(Intercept) 3.686819 0.07748442 3149 47.58142       0

Standardized Within-Group Residuals:
        Min          Q1         Med          Q3         Max 
-1.35596518 -1.15980953  0.02037839  0.93794078  1.59287762 

Number of Observations: 3168
Number of Groups: 19 

### m1 is a multilevel model with random intercept for states

In [44]:
m1 <- lme(pid7 ~ age + female + as.factor(race) + college  + authoritarianism + as.factor(race) * authoritarianism,
           data = df1,
           method = "ML", 
           na.action = "na.omit",
           random = ~ 1|state)
summary(m1)



Linear mixed-effects model fit by maximum likelihood
  Data: df1 
       AIC      BIC    logLik
  13469.69 13536.36 -6723.844

Random effects:
 Formula: ~1 | state
        (Intercept) Residual
StdDev:   0.1760153 2.017592

Fixed effects:  pid7 ~ age + female + as.factor(race) + college + authoritarianism +      as.factor(race) * authoritarianism 
                                       Value Std.Error   DF   t-value p-value
(Intercept)                        2.7494522 0.1514368 3141 18.155772  0.0000
age                                0.0116408 0.0021767 3141  5.348002  0.0000
female                            -0.2795868 0.0736664 3141 -3.795309  0.0002
as.factor(race)2                  -0.3623113 0.3921291 3141 -0.923959  0.3556
as.factor(race)3                  -0.0862119 0.1314702 3141 -0.655753  0.5120
college                           -0.1235605 0.0758171 3141 -1.629717  0.1033
authoritarianism                   2.3222825 0.1436991 3141 16.160726  0.0000
as.factor(race)2:authoritar

### m2 is a multilevel model with random slope for authoritrianism and random intercept for states

In [45]:
m2 <- lme(pid7 ~ age + female + as.factor(race) + college + authoritarianism + as.factor(race) * authoritarianism,
          data = df1,
          method = "ML", 
          na.action = "na.omit",
          random = ~ authoritarianism|state,
          control = lmeControl(opt = 'optim'))
summary(m2)



Linear mixed-effects model fit by maximum likelihood
  Data: df1 
       AIC      BIC    logLik
  13473.69 13552.48 -6723.846

Random effects:
 Formula: ~authoritarianism | state
 Structure: General positive-definite, Log-Cholesky parametrization
                 StdDev     Corr  
(Intercept)      0.17570343 (Intr)
authoritarianism 0.01003074 -0.006
Residual         2.01759642       

Fixed effects:  pid7 ~ age + female + as.factor(race) + college + authoritarianism +      as.factor(race) * authoritarianism 
                                       Value Std.Error   DF   t-value p-value
(Intercept)                        2.7494418 0.1514023 3141 18.159838  0.0000
age                                0.0116397 0.0021766 3141  5.347562  0.0000
female                            -0.2795866 0.0736666 3141 -3.795297  0.0002
as.factor(race)2                  -0.3623959 0.3921308 3141 -0.924171  0.3555
as.factor(race)3                  -0.0862955 0.1314700 3141 -0.656390  0.5116
college           

In [46]:
anova(m0, m1, m2)



,call,Model,df,AIC,BIC,logLik,Test,L.Ratio,p-value
m0,"lme.formula(fixed = pid7 ~ 1, data = df1, random = ~1 | state, method = ""ML"", na.action = ""na.omit"")",1,3,13942.80,13960.99,-6968.402,,NA,NA
m1,"lme.formula(fixed = pid7 ~ age + female + as.factor(race) + college + authoritarianism + as.factor(race) * authoritarianism, data = df1, random = ~1 | state, method = ""ML"", na.action = ""na.omit"")",2,11,13469.69,13536.36,-6723.844,1 vs 2,4.891152e+02,1.521653e-100
m2,"lme.formula(fixed = pid7 ~ age + female + as.factor(race) + college + authoritarianism + as.factor(race) * authoritarianism, data = df1, random = ~authoritarianism | state, method = ""ML"", na.action = ""na.omit"", control = lmeControl(opt = ""optim""))",3,13,13473.69,13552.48,-6723.846,2 vs 3,4.339583e-03,9.978326e-01


### the p value for m0 vs m1 is nearly to 0, which indicates, the model with random intercept performs much better than m0; the p value for m1 vs m2 is 0.998, which indicates the m2 does not performs better than m1; therefore, m1 should be the best model 